### Depp Convolutional Neural Network

In this example, we are going to construct a network that looks like:

![Convolutional neural network](../images/mnist-convnet.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

### The Network class

In [ ]:
class Convnet(nn.Module):
    
    def __init__(self):
        super(Convnet, self).__init__()
        pass
    
    def forward(self, x):
        return x
    
    def _flatten(self, x):
        pass

### importing the data